In [2]:
import pandas as pd
# import csv files, combine and drop columns with NA values
dfs = []
for i in range(1, 5):
 dfs.append(pd.read_csv(f"./dblp-{i}.csv"))

df = pd.concat(dfs)
df = df.rename(columns={"Unnamed: 0": "id"})
df = df.dropna(axis=1)



In [3]:
# add two columns for further usage 
df['author'] = 'author'
df['title'] = 'title'


There are authors placed in the title column, so I'm trying to replace the values in the columns that I made above with the right one (authors in author-column, and title in tile-column)
I figured that authors are separated with a | so I'm looking for this charachter, and for cases were there's only one author, I assume that an author's name isn't longer than 4 strings.

In [10]:
# try to store authors and titles in right columns

for index, row in df.iterrows():
    if (r'|' in row['ptitle']) and len(row['ptitle']) < 5:
            # df.loc[index, 'title'] = row['pauthor']
            df.loc['title'] = df.loc['pauthor']
            # df.loc[index, 'author'] = row['ptitle']
            df.loc['author'] = df.loc['ptitle']
    else:
        # df.loc[index, 'title'] = row['ptitle']
        df.loc['title'] = df.loc['ptitle']
        # df.loc[index, 'author'] = row['pauthor']
        df.loc['author'] = df.loc['pauthor']

# remove former columns 
df.drop([df['pauthor'], df['ptitle']], axis=1)

KeyError: 'ptitle'

Another attempt to replace the values, yet again unsuccesful

In [ ]:
# tokenize in order to be able to check the number of strings in a cell (by len(title) < 5)

from nltk import word_tokenize 
df['ptitle'] = df['ptitle'].apply(word_tokenize)
df['pauthor'] = df['pauthor'].apply(word_tokenize)

df['title'] = 'title'
df['author'] = 'author'

for ptitle, title, pauthor, author in zip(df['ptitle'],df['title'], df['pauthor'], df['author']):
    for char in ptitle:
        if (r'|' in char):
            title = pauthor
            author = ptitle 
        else:
            title = ptitle 
            author = pauthor 
    if len(title) < 5:
        title = pauthor
        author = ptitle
    else:
        title = ptitle
        author = pauthor
    
print(df)


# from nltk import word_tokenize 
# import re
# # df['ptitle'] = df['ptitle'].apply(word_tokenize)
# # df['pauthor'] = df['pauthor'].apply(word_tokenize)


# for title, author in zip(df['ptitle'],df['pauthor']):
#     # title = df['ptitle']
#     # author = df['pauthor']
#     regex = re.compile('|')
#     # char = '|'
#     for tokens in title: 
#         if (regex.search(tokens) != None) or (len(tokens) < 5):
#             # if (r'|' in tokens) and (len(tokens) < 5):
#             correctauthor = title
#             correcttitle = author
#             author = correctauthor
#             title = correcttitle
#         else:
#             title = title
#             author = author



Preprocess titles in order to be able to compare them easier with eachother

In [8]:
import preprocess as pp
from nltk import word_tokenize 

df['title_cleaned'] = df['ptitle'].apply(pp.string_cleaner)
df['title_token'] = df['title_cleaned'].apply(word_tokenize)
df['title_lemma'] = df['title_token'].apply(pp.lemmatize)
df['lemma_text'] = df['title_lemma'].apply(lambda w: ' '.join(w)) 
df['pos_year'] = df['pyear'].apply(abs)

df.drop(['title_lemma', 'title_token', 'title_cleaned', 'ptitle', 'pyear'], axis=1)

    

,id,pauthor,pid,pkey,ptype_id,pjournal_id,pbooktitle_id,pjournalfull_id,pbooktitlefull_id,partition,lemma_text,pos_year
0,4,Jorge Semião|Juan J. Rodríguez-Andina|Fabian V...,180843,conf/dft/SemiaoRVSTT07,1,0,4,0,4,1,improving the tolerance of pipeline based circ...,2007
1,7,Patrice Caire,162991,conf/dagstuhl/Caire07,2,0,7,0,7,1,a normative multi agent system approach to the...,2007
2,10,Sundeep B|Andrew Thangaraj,2261406,journals/tit/BT07,0,2,9,2,9,1,self orthogonality of q ary image of qm ary co...,2007
3,18,Gerardo Pardo-Castellote,349720,conf/icdcsw/Pardo-Castellote03,1,0,11,0,11,1,omg data distribution service architectural ov...,2003
4,19,Ki-Hoon Lee|Kyu-Young Whang|Wook-Shin Han|Min-...,1922328,journals/corr/abs-0911-4329,3,5,5,5,5,1,structural consistency enabling xml keyword se...,2009
...,...,...,...,...,...,...,...,...,...,...,...,...
4251,9975,Online bin packing with arbitrary release times.,2245299,journals/tcs/ShiY08,0,12,5,12,5,4,yongqiang shideshi ye,2008
4252,9978,Chew Lim Tan|Henry Wai Kit Chia,460233,conf/ijcai/TanC01,2,0,71,0,71,4,neural logic network learning using genetic pr...,2001
4253,9981,Sue Newell|Jacky Swan|Joseph Weiss,289610,conf/hicss/NewellSW04,1,0,295,0,295,4,project management minitrack introduction,2004
4254,9988,Martijn Hendriks|Barend van den Nieuwelaar|Fri...,2224414,journals/sttt/HendriksNV06,3,374,5,374,5,4,model checker aided design of a controller for...,2006
